In [20]:
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
import nltk

In [3]:
arxiv_df = pd.read_parquet("data/query_result.parquet")

In [41]:
# Function to split text into title, abstract, and body
def split_text(text_list):
    title = text_list[0][0]  # First list, first element
    abstract = text_list[1][0]  # Second list, first element
    body = " ".join(
        [" ".join(paragraph) for paragraph in text_list[2:]]
    )  # Join all subsequent lists as body

    return title, abstract, body


# Apply function to create new columns
new_arxiv_df = arxiv_df.copy()
new_arxiv_df[["title", "abstract", "body"]] = arxiv_df["text"].apply(
    lambda x: pd.Series(split_text(x))
)

# Drop the old 'text' column if no longer needed
new_arxiv_df = new_arxiv_df.drop(columns=["text"])

In [43]:
# List of keywords related to machine learning and deep learning
keywords = [
    "machine learning",
    "neural networks",
    "deep learning",
    "convolutional neural networks",
    "reinforcement learning",
    "supervised learning",
    "unsupervised learning",
    "transfer learning",
    "generative models",
    "natural language processing",
    "computer vision",
    "speech recognition",
]

# Filter the DataFrame by checking if any keyword is in the 'abstract' or 'title' column
filtered_df = new_arxiv_df[
    new_arxiv_df["title"].str.contains("|".join(keywords), case=False, na=False)
    | new_arxiv_df["abstract"].str.contains("|".join(keywords), case=False, na=False)
]

In [45]:
filtered_df.to_parquet('./filtered_df_arxiv.parquet')